In [1]:
import time
import pandas as pd
import json
import urllib
import requests as rq
import base64
from urllib.parse import quote, urlencode

In [51]:
API_key = ""
SECRET = ""
url = "https://api.idealista.com/oauth/token"

In [52]:
def get_oauth_token():
    url = "https://api.idealista.com/oauth/token"
    apikey= 'lzif3ym4qh4999ek0zti5c9of7wgk4yt' #sent by idealista
    secret= 'DzHIKOp1DUN3'  #sent by idealista
    encoded_api_key = quote(apikey, safe='')
    encoded_api_secret = quote(secret, safe='')
    enc = f"{encoded_api_key}:{encoded_api_secret}"
    auth = base64.b64encode(enc.encode())
    headers = {'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8' ,'Authorization' : 'Basic ' + auth.decode()}
    params = urlencode({'grant_type':'client_credentials'})
    content = rq.post(url,headers = headers, params=params)
    bearer_token = json.loads(content.text)['access_token']
    return bearer_token

In [53]:
def search_api(token, url):
    headers = {'Content-Type': 'Content-Type: multipart/form-data;', 'Authorization' : 'Bearer ' + token}
    content = rq.post(url, headers = headers)
    result = json.loads(content.text)
    return result

In [54]:
#create new empty dataframe
df_tot = pd.DataFrame()

In [55]:
#iterate over different pages of the result, get the request result, and save houses' data on a df and add to the total df 

#edit the following three according to your needs
first_page=81
last_page=90
locationId = '0-EU-PT-11-06' #lisboa

#query variables
country = 'pt' #values: es, it, pt
language = 'pt'
max_items = '50'
operation = 'sale'
property_type = 'homes'
order = 'publicationDate'
sort = 'desc'



for i in range(first_page,last_page+1):

    pagina = str(i)
    
    url = ('https://api.idealista.com/3.5/'+country+'/search?operation='+operation+
               '&maxItems='+max_items+
               '&order='+order+
               '&locationId='+locationId+
               '&propertyType='+property_type+
               '&sort='+sort+
               '&numPage='+pagina+
               '&language='+language)
    
    oauth_token = get_oauth_token()
    current_data = search_api(oauth_token, url)

    # Save current_data to disk
    with open(f'collected_data/current_data_page_{pagina}.json', 'w') as json_file:
        json.dump(current_data, json_file)


    
    #add current data to the total data of this session
    current_data_df = pd.DataFrame.from_dict(current_data['elementList'])
    df_tot = pd.concat([df_tot,current_data_df]) 



    time.sleep(1)


#save df to csv
df_tot.to_csv(f'collected_data/lisboa_p{first_page}-{last_page}.csv', index=False)

In [56]:
#check df total
print(f'Shape of df_tot: {df_tot.shape}')
df_tot.head()

Shape of df_tot: (500, 39)


,propertyCode,thumbnail,externalReference,numPhotos,floor,price,priceInfo,propertyType,operation,size,...,hasPlan,has3DTour,has360,hasStaging,highlight,topNewDevelopment,topPlus,parkingSpace,newDevelopmentFinished,neighborhood
0,33290399,https://img3.idealista.pt/blur/WEB_LISTING/0/i...,2024-08,24,bj,260000.0,"{'price': {'amount': 260000.0, 'priceDropInfo'...",flat,sale,87.0,...,True,False,False,False,{'groupDescription': 'Top'},False,False,NaN,NaN,NaN
1,33287555,https://img3.idealista.pt/blur/WEB_LISTING/0/i...,PP174516,27,NaN,325000.0,{'price': {'amount': 325000.0}},flat,sale,67.0,...,False,False,False,False,NaN,False,False,NaN,NaN,NaN
2,33290423,https://img3.idealista.pt/blur/WEB_LISTING/0/i...,ZMPT565698,34,15,1100000.0,{'price': {'amount': 1100000.0}},flat,sale,200.0,...,True,False,False,False,NaN,False,False,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",NaN,NaN
3,33286788,https://img3.idealista.pt/blur/WEB_LISTING/0/i...,NaN,21,1,200000.0,"{'price': {'amount': 200000.0, 'priceDropInfo'...",flat,sale,52.0,...,False,False,False,False,NaN,False,False,NaN,NaN,NaN
4,33290366,https://img3.idealista.pt/blur/WEB_LISTING/0/i...,121701109-1328,20,7,490000.0,{'price': {'amount': 490000.0}},flat,sale,98.0,...,False,False,False,False,NaN,False,False,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",NaN,NaN
